# Hugging Face Text Generation Inference available for AWS Inferentia2

Text Generation Inference (TGI), is a purpose-built solution for deploying and serving Large Language Models (LLMs) for production workloads at scale. TGI enables high-performance text generation using Tensor Parallelism and continuous batching for the most popular open LLMs, including Llama, Mistral, and more. Text Generation Inference is used in production by companies such as Grammarly, Uber, Deutsche Telekom, and many more.

The integration of TGI into Amazon SageMaker, in combination with AWS Inferentia2, presents a powerful solution and viable alternative to GPUs for building production LLM applications. The seamless integration ensures easy deployment and maintenance of models, making LLMs more accessible and scalable for a wide range of production use cases.

With the new TGI for AWS Inferentia2 on Amazon SageMaker, AWS customers can benefit from the same technologies that power highly-concurrent, low-latency LLM experiences like HuggingChat, OpenAssistant, and Serverless Endpoints for LLMs on the Hugging Face Hub.


## Deploy Mistral-7b on AWS Inferentia2 using Amazon SageMaker

### Description
Mixtral 8x7B Instruct
License
This model has Apache 2.0 License. Please read the terms carefully.

The Mixtral-8x7B Large Language Model (LLM) is a pretrained generative Sparse Mixture of Experts developed by Mistral AI. Mixtral has the following capabilities:

It gracefully handles a context of 32k tokens.
It handles English, French, Italian, German and Spanish.
It shows strong performance in code generation.
It has both a pre-trained model and a fine-tuned instruction-following model.


This is the repository for the 7B fine-tuned model, optimized for dialogue use cases and converted for the Hugging Face Transformers format. Links to other models can be found in the index at the bottom. This demo is running on AWS Inferentia2,

The deployed model can be used for running inference on any input text. Example python code for how to run inference on the deployed model is given in a notebook.

We are going to show you how to:

1. Setup development environment
2. Retrieve the TGI Neuronx Image
3. Deploy Mistral-7B to Amazon SageMaker
4. Run inference and chat with the model

Let’s get started.

### 1. Setup development environment
We are going to use the sagemaker python SDK to deploy Mistral-7B to Amazon SageMaker. We need to make sure to have an AWS account configured and the sagemaker python SDK installed.
You need access to an IAM Role with the required permissions for Sagemaker. You can find out more about it [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

In [1]:
!pip install transformers "sagemaker>=2.206.0" --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.68 which is incompatible.
amazon-sagemaker-jupyter-scheduler 3.0.7 requires jupyter-scheduler==2.5, but you have jupyter-scheduler 2.5.1 which is incompatible.
autogluon-multimodal 0.8.2 requires transformers[sentencepiece]<4.32.0,>=4.31.0, but you have transformers 4.39.0 which is incompatible.


In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it doesn't exist
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::850751315356:role/service-role/AmazonSageMaker-ExecutionRole-20240322T092692
sagemaker session region: us-east-2


### 2. Retrieve TGI Neuronx Image

The new Hugging Face TGI Neuronx DLCs can be used to run inference on AWS Inferentia2. You can use the get_huggingface_llm_image_uri method of the sagemaker SDK to retrieve the appropriate Hugging Face TGI Neuronx DLC URI based on your desired backend, session, region, and version. You can find all the available versions [here](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true).


In [3]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface-neuronx",
  version="0.0.20"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")


llm image uri: 763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-tgi-inference:1.13.1-optimum0.0.20-neuronx-py310-ubuntu22.04


## Text Generation Interface

Text Generation Inference (TGI) on Inferentia2 supports popular open LLMs, including Llama, Mistral, and more. You can check the full list of supported models (text-generation) [here](https://huggingface.co/docs/optimum-neuron/package_reference/export#supported-architectures).

You can find detailed information about the base model on its [Model Card](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2).

compilation arguments

{
  "num_cores": 2,
  "auto_cast_type": "fp16"
}

input_shapes

{
  "sequence_length": 2048,
  "batch_size": 1
}

In [4]:
import json
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config & model config
instance_type = "ml.inf2.8xlarge"
health_check_timeout = 900
batch_size = 1
sequence_length = 2048

# Define Model and Endpoint configuration parameter
config = {
  # 'HF_MODEL_ID': "aws-neuron/Llama-2-7b-chat-hf-seqlen-2048-bs-1",
  'HF_MODEL_ID': "aws-neuron/Mistral-7B-Instruct-v0.2-seqlen-2048-bs-1-cores-2",
  'MAX_CONCURRENT_REQUESTS': json.dumps(batch_size),
  'MAX_INPUT_LENGTH': json.dumps(1024), 
    # ArgumentValidation("`max_batch_prefill_tokens` must be >= `max_input_length`.")
  'MAX_TOTAL_TOKENS': json.dumps(sequence_length),
  'MAX_BATCH_PREFILL_TOKENS': json.dumps(int(sequence_length*batch_size / 2)),
  'MAX_BATCH_TOTAL_TOKENS': json.dumps(sequence_length*batch_size),
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

### 4. Deploy Mistral-7B to Amazon SageMaker
Text Generation Inference (TGI) on Inferentia2 supports popular open LLMs, including Llama, Mistral, and more. You can check the full list of supported models (text-generation) here.

### Compiling LLMs for Inferentia2

At the time of writing, AWS Inferentia2 does not support dynamic shapes for inference, which means that we need to specify our sequence length and batch size ahead of time. To make it easier for customers to utilize the full power of Inferentia2, we created a neuron model cache, which contains pre-compiled configurations for the most popular LLMs. A cached configuration is defined through a model architecture (Mistral), model size (7B), neuron version (2.16), number of inferentia cores (2), batch size (2), and sequence length (2048).

This means compiling fine-tuned checkpoints for Mistral 7B with the same configuration will take only a few minutes. Examples of this are mistralai/Mistral-7B-v0.1 and HuggingFaceH4/zephyr-7b-beta.

In [5]:
# Deploy model to an endpoint
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
)

Your model is not compiled. Please compile your model before using Inferentia.


----------------------!

In [138]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def call_mistral(question, tone, n_tries=10):
    ask = question if question else "Tell me something interesting!"
    if n_tries == 0:
        print("I am not interested in answering your request.")
        return None
    # Generation arguments
    payload = {
        "do_sample": False,
        "top_p": 0.6,
        "temperature": 0.9,
        "top_k": 50,
        "max_new_tokens": 512,
        "repetition_penalty": 1.03,
        "return_full_text": False,
        "stop": ["</s>"]
    }
    prompt = f'<s>[INST] {tones[tone]} {ask} [/INST]</s>'

    chat = llm.predict({"inputs":prompt, "parameters":payload})
    t = chat[0]["generated_text"][len(prompt):]
    # if t == '':
    #     t = call_mistral(question, tone=tone, n_tries=n_tries-1)
    print(t)


tones = {'Short': 'Answer any question with one small sentence.',
         'Neutral': 'You are a helpful assistant.',
        'For kids': 'Answer any question as if it was asked by a three-year old kid and you were a teacher trying to give an understandable answer.', 
        'French': 'Answer the following question using French language:',
        'Pirate': 'Answer any question as if you were Jack Sparrow, from "Pirates of the Caribbean", but never say who you are.',
        'Expert': 'Answer the following question as if you are a software engineer, but never say that you are'
         }

tone_list = [t for t in tones]

tone = widgets.RadioButtons(
    options=tone_list,
    value=tone_list[0],
    description='Tone:',
    disabled=False,
    continuous_update=False
)

q_input = widgets.Textarea(
    value='',
    placeholder='Hi there! How can I help you?',
    description='Question:',
    disabled=False,
    continuous_update=False,
    layout={'width': '70%'}
)

out = widgets.interactive_output(call_mistral, {'question': q_input, 'tone': tone})

In [139]:
widgets.VBox([widgets.HBox([q_input, tone]), out])